## @property

In [ ]:
class Temperature:
    def __init__(self, celsius=0):
        self._celsius = celsius

    @property
    def celsius(self):
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Temperature below absolute zero!")
        self._celsius = value

    @property
    def fahrenheit(self):
        return self._celsius * 9/5 + 32

    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = (value - 32) * 5/9


In [ ]:
# Usage
temp = Temperature()
temp.celsius = 25  # Clean attribute-like access with validation
print(f"{temp.celsius}°C = {temp.fahrenheit}°F")

25°C = 77.0°F


## @cached_property

In [ ]:
from functools import cached_property
import time

class DataAnalyzer:
    def __init__(self, dataset):
        self.dataset = dataset

    @cached_property
    def complex_analysis(self):
        print("Running expensive analysis...")
        time.sleep(2)  # Simulating heavy computation
        return sum(x**2 for x in self.dataset)


In [ ]:
# Usage
analyzer = DataAnalyzer(range(1000000))
print("First access:")
t1 = time.time()
result1 = analyzer.complex_analysis
t2 = time.time()
print(f"Result: {result1}, Time: {t2-t1:.2f}s")

print("\nSecond access:")
t1 = time.time()
result2 = analyzer.complex_analysis
t2 = time.time()
print(f"Result: {result2}, Time: {t2-t1:.2f}s")

First access:
Running expensive analysis...
Result: 333332833333500000, Time: 2.23s

Second access:
Result: 333332833333500000, Time: 0.00s


## @lru_cache

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

# Watch how fast this runs compared to non-cached version
import time
start = time.time()
result = fibonacci(35)
end = time.time()
print(f"Fibonacci(35) = {result}, calculated in {end-start:.6f} seconds")

# Check cache statistics
print(f"Cache info: {fibonacci.cache_info()}")

Fibonacci(35) = 9227465, calculated in 0.000108 seconds
Cache info: CacheInfo(hits=33, misses=36, maxsize=128, currsize=36)


## @contextlib.contextmanager

In [ ]:
from contextlib import contextmanager

@contextmanager
def file_manager(filename, mode):
    try:
        f = open(filename, mode)
        yield f
    finally:
        f.close()

@contextmanager
def timer():
    import time
    start = time.time()
    yield
    elapsed = time.time() - start
    print(f"Elapsed time: {elapsed:.6f} seconds")


In [ ]:
# Usage
with file_manager('test.txt', 'w') as f:
    f.write('Hello, context managers!')

with timer():
    # Code to time
    sum(i*i for i in range(1000000))

Elapsed time: 0.242454 seconds


## @functools.singledispatch

In [ ]:
from functools import singledispatch
from datetime import date, datetime

@singledispatch
def format_output(obj):
    return str(obj)

@format_output.register
def _(obj: int):
    return f"INTEGER: {obj:+d}"

@format_output.register
def _(obj: float):
    return f"FLOAT: {obj:.2f}"

@format_output.register
def _(obj: date):
    return f"DATE: {obj.strftime('%Y-%m-%d')}"

@format_output.register(list)
def _(obj):
    return f"LIST: {', '.join(format_output(x) for x in obj)}"

In [ ]:
# Usage
results = [
    format_output("Hello"),
    format_output(42),
    format_output(-3.14159),
    format_output(date(2025, 2, 21)),
    format_output([1, 2.5, "three"])
]

for r in results:
    print(r)

Hello
INTEGER: +42
FLOAT: -3.14
DATE: 2025-02-21
LIST: INTEGER: +1, FLOAT: 2.50, three


## @functools.total_ordering

In [ ]:
from functools import total_ordering

@total_ordering
class Version:
    def __init__(self, major, minor, patch):
        self.major = major
        self.minor = minor
        self.patch = patch

    def __eq__(self, other):
        if not isinstance(other, Version):
            return NotImplemented
        return (self.major, self.minor, self.patch) == (other.major, other.minor, other.patch)

    def __lt__(self, other):
        if not isinstance(other, Version):
            return NotImplemented
        return (self.major, self.minor, self.patch) < (other.major, other.minor, other.patch)

    def __repr__(self):
        return f"v{self.major}.{self.minor}.{self.patch}"

In [ ]:
# Usage
versions = [
    Version(2, 0, 0),
    Version(1, 9, 5),
    Version(1, 11, 0),
    Version(2, 0, 1)
]

print(f"Sorted versions: {sorted(versions)}")
print(f"v1.9.5 > v1.11.0: {Version(1, 9, 5) > Version(1, 11, 0)}")
print(f"v2.0.0 >= v2.0.0: {Version(2, 0, 0) >= Version(2, 0, 0)}")
print(f"v2.0.1 <= v2.0.0: {Version(2, 0, 1) <= Version(2, 0, 0)}")

Sorted versions: [v1.9.5, v1.11.0, v2.0.0, v2.0.1]
v1.9.5 > v1.11.0: False
v2.0.0 >= v2.0.0: True
v2.0.1 <= v2.0.0: False


## @functools.wraps

In [ ]:
import functools

def log_execution(func):
    @functools.wraps(func)  # Preserves func's name, docstring, etc.
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args: {args}, kwargs: {kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned: {result}")
        return result
    return wrapper

@log_execution
def add(a, b):
    """Add two numbers and return the result."""
    return a + b

# Without @wraps, help(add) would show wrapper's info
help(add)  # Shows the original docstring
print(f"Function name: {add.__name__}")  # Shows "add", not "wrapper"

result = add(5, 3)

Help on function add in module __main__:

add(a, b)
    Add two numbers and return the result.

Function name: add
Calling add with args: (5, 3), kwargs: {}
add returned: 8
